# Analysis of Burnt Areas by County in a State

This script anlayses the burnt areas caused by wildfires in a specific state. The script includes:

1. **Retrieving County Boundaries**: Using a GeoJSON API to retrieve county boundaries for the selected state.
2. **Retrieving Fire Perimeters**: Retrieving fire perimeter data from an external API.
3. **Calculating Burnt Area by County**: Overlaying fire perimeters on county boundaries to calculate the burnt area for each county.
4. **summarising Results**: Generating a summary table of burnt areas by county and saving it as a CSV file.

The analysis is performed for a specified state which can be changed at the bottom of the script, and the results include both table and map outputs.

In [14]:
#Install necessary libraries, uncomment if not already installed
#!pip install requests geopandas pandas matplotlib

#Import necessary libraries
import requests
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

## County Data Retrieval

This function retrieves county boundary data for a specified state from the ArcGIS REST service.

In [15]:
def get_counties_for_state(state_name):
    """
    Gets all counties for a specific state.
    
    Args:
        state_name (str): The name of the state to get counties for.
        
    Returns:
        GeoDataFrame: A GeoDataFrame containing all counties in the state.
    """
    counties_url = "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Counties_Generalized_Boundaries/FeatureServer/0/query"
    
    #Set up the request
    counties_params = {
        'where': f"STATE_NAME = '{state_name}'",
        'outFields': '*',
        'returnGeometry': 'true',
        'f': 'geojson'
    }
    
    #Get the data from the server
    counties_response = requests.get(counties_url, params=counties_params)
    counties_geojson = counties_response.json()
    counties_gdf = gpd.GeoDataFrame.from_features(counties_geojson['features'])
    return counties_gdf

## Fire Data Retrieval

This function retrieves wildfire perimeter data from the WFIGS Interagency service. I ran into an issue where only 2000 perimeters were being returned so to get around this I used batch processing to retrieve all perimeters, as there may be more than 2000.

In [16]:
def get_fire_perimeters():
    """
    Gets all fire perimeters by making multiple requests.
    
    Returns:
        GeoDataFrame: A GeoDataFrame containing fire perimeters.
    """
    fires_url = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/WFIGS_Interagency_Perimeters_YearToDate/FeatureServer/0/query"
    
    #Store all features we find
    all_features = []
    
    #Start at the beginning
    offset = 0
    batch_size = 2000
    
    print("Fetching fire perimeters in batches...")
    
    #Loop until we get all fire perimeters
    while True:
        #Set up the request for this batch
        fires_params = {
            'where': '1=1',
            'outFields': 'OBJECTID,poly_IncidentName,poly_Acres_AutoCalc',
            'returnGeometry': 'true',
            'resultRecordCount': batch_size,
            'resultOffset': offset,
            'f': 'geojson'
        }
        
        #Get the data from the server
        fires_response = requests.get(fires_url, params=fires_params)
        fires_geojson = fires_response.json()
        
        #Check if we got any features
        if 'features' not in fires_geojson or len(fires_geojson['features']) == 0:
            break
        
        #Add the features to our list
        features_in_batch = fires_geojson['features']
        all_features.extend(features_in_batch)
        
        print(f"Retrieved batch of {len(features_in_batch)} fire perimeters (total so far: {len(all_features)})")
        
        #Check if we've reached the end
        if len(features_in_batch) < batch_size:
            break
        
        #Move to the next batch
        offset += batch_size
    
    #Create a GeoDataFrame from all the features
    if all_features:
        #Make a valid GeoJSON with all features
        complete_geojson = {
            'type': 'FeatureCollection',
            'features': all_features
        }
        
        fires_gdf = gpd.GeoDataFrame.from_features(complete_geojson)
        print(f"Total fire perimeters loaded: {len(fires_gdf)}")
        return fires_gdf
    else:
        print("No fire perimeters were found.")
        return gpd.GeoDataFrame()

## Spatial Analysis

This function performs the core spatial analysis to calculate burnt area by county. It intersects county boundaries with fire perimeters and calculates the proportion of each fire that falls within each county.

In [17]:
def calculate_burnt_area_by_county(counties_gdf, fires_gdf):
    """
    Calculates the burnt area for each county.
    
    Args:
        counties_gdf (GeoDataFrame): GeoDataFrame containing county boundaries
        fires_gdf (GeoDataFrame): GeoDataFrame containing fire perimeters
        
    Returns:
        DataFrame: A DataFrame with burnt area calculations by county.
    """
    #Check if we have fire data
    if fires_gdf.empty:
        print("No fire perimeters found for analysis.")
        return pd.DataFrame(columns=['County', 'FIPS', 'Burnt Area (Acres)'])
    
    #Make sure the coordinate systems match
    if counties_gdf.crs != fires_gdf.crs and fires_gdf.crs is not None:
        print("Converting coordinate systems to match...")
        counties_gdf = counties_gdf.to_crs(fires_gdf.crs)
    
    #Create a list to store our results
    results = []
    
    print("Calculating fire area by county...")
    print(f"Measuring {len(counties_gdf)} counties against {len(fires_gdf)} fire perimeters.")
    
    #Look at each county
    for county_row in counties_gdf.itertuples():
        county_name = county_row.NAME
        county_geom = county_row.geometry
        
        #Find fires that overlap with this county
        overlapping_fires = fires_gdf[fires_gdf.intersects(county_geom)]
        
        #Calculate total burnt acres for this county
        total_burnt_acres = 0
        
        #If there are overlapping fires
        if not overlapping_fires.empty:
            for fire_row in overlapping_fires.itertuples():
                fire_geom = fire_row.geometry
                
                #Find where the fire and county overlap
                intersection = county_geom.intersection(fire_geom)
                
                #Calculate what percentage of the fire is in this county
                if not intersection.is_empty:
                    proportion = intersection.area / fire_geom.area
                    burnt_acres = fire_row.poly_Acres_AutoCalc * proportion
                    total_burnt_acres += burnt_acres
        
        #Add this county's results to our list
        results.append({
            'County': county_name,
            'Burnt Area (Acres)': round(total_burnt_acres, 2)
        })
    
    #Create a DataFrame and sort by burnt area
    results_df = pd.DataFrame(results).sort_values('Burnt Area (Acres)', ascending=False)
    return results_df

## Analysis Function

This function coordinates the complete analysis process for a specified state. It retrieves county and fire data, calculates burnt area by county, saves the results to a CSV file, and returns both the results and geographic data for mapping.

In [18]:
def summarise_burnt_area_for_state(state_name):
    """
    Creates a summary of burnt area by county for a specified state.
    
    Args:
        state_name (str): The name of the state to anlayse
        
    Returns:
        tuple: A tuple containing (results_df, counties_gdf) where:
            - results_df is a DataFrame with burnt area by county
            - counties_gdf is the GeoDataFrame with county boundaries
    """
    print(f"Starting analysis for {state_name}...")
    
    #Get county boundaries
    counties_gdf = get_counties_for_state(state_name)
    
    #Get fire perimeters
    fires_gdf = get_fire_perimeters()
    
    #Calculate burnt area by county
    results_df = calculate_burnt_area_by_county(counties_gdf, fires_gdf)
    
    #Save results to a CSV file
    output_file = f"burnt_area_by_county_{state_name.replace(' ', '_')}.csv"
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    #Return both the results DataFrame and the counties GeoDataFrame for mapping
    return results_df, counties_gdf

## Run the Analysis

The code below allows you to analyse burnt area for any state. Change the state name variable to run the analysis for a different state. The results will be displayed as a table and map.

In [19]:
#1. Change your state here
state_name = "California"

#2. Run the analysis
burnt_area_df, counties_gdf = summarise_burnt_area_for_state(state_name)

#3. Display the pandas table
print(burnt_area_df)

Starting analysis for California...
Fetching fire perimeters in batches...
Retrieved batch of 2000 fire perimeters (total so far: 2000)
Retrieved batch of 270 fire perimeters (total so far: 2270)
Total fire perimeters loaded: 2270
Calculating fire area by county...
Measuring 58 counties against 2270 fire perimeters.
Results saved to burnt_area_by_county_California.csv
                    County  Burnt Area (Acres)
18      Los Angeles County            49289.61
36        San Diego County             6631.45
55          Ventura County              688.17
12         Imperial County               17.31
57             Yuba County               14.94
35   San Bernardino County               14.21
14             Kern County               11.45
13             Inyo County               10.53
39  San Luis Obispo County                3.95
31           Plumas County                1.74
32        Riverside County                1.58
30           Placer County                0.24
7         Del Nort